In [32]:
import pandas as pd
import numpy as np
from impyute.imputation.cs import fast_knn

ModuleNotFoundError: No module named 'impyute'

In [6]:
df = pd.read_excel('NAFLD with FIB4 and NFS Scores.xlsx').drop(294)
df

,Patient No.,Age,"Gender (Female=1, Male=2)",Height,Weight,Body Mass Index,Waist Circumference,Hip Circumference,Systolic Blood Pressure,Diastolic Blood Pressure,...,NAS score according to Kleiner,"NAS score>=4 and Fibrosis>=2 (If NAS score>=4 and Fibrosis>=2 = 1, If not = 0)","Fibrosis status (No=0, Yes=1) (Fibrosis 1 and above, there is Fibrosis)","Significant Fibrosis (No=0, Yes=1) (If Fibrosis 2 and above, there is Significant Fibrosis)","Advanced Fibrosis (No=0, Yes=1) (If Fibrosis is 3 and above, there is Advanced Fibrosis)","Cirrhosis (No=0, Yes=1) (If Fibrosis is 4, there is Cirrhosis)","Diagnosis according to SAF (NASH=1, NAFL=2)","Type of Disease (Mild illness=1, Severe illness=2)",FIB-4,NFS
0,1,60,1,150,80,35.56,119.0,104.0,135,77,...,4,1,1,0,0,0,1,2,1.066491,-0.557850
1,2,55,1,154,73,30.78,101.0,98.0,120,72,...,4,1,0,0,0,0,1,2,0.801388,-2.290680
2,3,53,2,170,101,34.95,116.0,112.0,120,80,...,4,1,1,1,1,0,1,2,1.532768,-0.647403
3,4,33,2,175,95,31.02,114.0,114.0,124,83,...,4,1,1,0,0,0,1,2,0.352520,-3.661870
4,5,23,2,162,68,25.91,101.0,97.0,124,79,...,4,1,0,0,0,0,1,2,0.490765,-3.995284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,601,58,1,150,85,37.78,110.0,108.0,140,100,...,6,0,1,0,0,0,1,2,1.890277,-2.030823
601,602,63,2,186,99,28.61,108.0,117.0,120,80,...,6,0,1,0,0,0,1,2,1.136866,-1.607660
602,603,44,1,159,102,40.30,127.0,108.0,150,80,...,6,0,1,1,1,0,1,2,0.541972,-2.295062
603,604,44,1,159,102,40.35,127.0,108.0,150,80,...,6,0,1,1,1,0,1,2,0.541972,-2.270562


In [14]:
X = pd.concat([df.iloc[:,1:-13], df[['FIB-4', 'NFS']]], axis=1)
X = X.drop(['Height', 'Weight'], axis=1)
X

,Age,"Gender (Female=1, Male=2)",Body Mass Index,Waist Circumference,Hip Circumference,Systolic Blood Pressure,Diastolic Blood Pressure,"Diyabetes Mellitus (No=0, Yes=1)","Hypertension (No=0, Yes=1)","Hyperlipidemia (No=0, Yes=1)",...,Glucose,Insulin,HOMA,"Insulin resistance according to HOMA (insulin resistance = 1, no insulin resistance = 2)",Uric Acid,BUN,Creatinine,Hemoglobin - A1C,FIB-4,NFS
0,60,1,35.56,119.0,104.0,135,77,0,1,1,...,119.0,29.20,8.58,1.0,NaN,12.0,0.58,NaN,1.066491,-0.557850
1,55,1,30.78,101.0,98.0,120,72,0,0,0,...,NaN,NaN,NaN,NaN,NaN,18.0,0.63,NaN,0.801388,-2.290680
2,53,2,34.95,116.0,112.0,120,80,1,1,1,...,93.0,NaN,NaN,NaN,NaN,NaN,NaN,4.70,1.532768,-0.647403
3,33,2,31.02,114.0,114.0,124,83,1,0,1,...,90.0,23.00,5.11,1.0,6.57,20.0,0.73,4.70,0.352520,-3.661870
4,23,2,25.91,101.0,97.0,124,79,0,0,0,...,89.0,NaN,NaN,NaN,NaN,NaN,NaN,4.82,0.490765,-3.995284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,58,1,37.78,110.0,108.0,140,100,0,1,1,...,91.0,17.56,3.95,1.0,7.60,11.0,0.82,5.50,1.890277,-2.030823
601,63,2,28.61,108.0,117.0,120,80,0,1,1,...,115.0,NaN,NaN,NaN,NaN,NaN,NaN,5.80,1.136866,-1.607660
602,44,1,40.30,127.0,108.0,150,80,1,0,1,...,133.0,NaN,NaN,NaN,NaN,NaN,NaN,6.60,0.541972,-2.295062
603,44,1,40.35,127.0,108.0,150,80,1,0,1,...,133.0,36.00,11.82,1.0,5.10,26.0,0.87,6.60,0.541972,-2.270562


In [12]:
adv_fib_y = pd.DataFrame(df.iloc[:, -6])
adv_fib_y.rename(columns={'Advanced Fibrosis (No=0, Yes=1) (If Fibrosis is 3 and above, there is Advanced Fibrosis)': 'Advanced Fibrosis'}, inplace=True)
adv_fib_y

,Advanced Fibrosis
0,0
1,0
2,1
3,0
4,0
...,...
600,0
601,0
602,1
603,1


In [29]:
X_columns_1 = X.isna().sum().sort_values(ascending=False)[-18:].index   # features with no missing values
X_columns_2 = X.isna().sum().sort_values(ascending=False)[15:].index    # features <= 25% missing values threshold
X_baseline_1 = X[X_columns_1]   # Data Frame of features with no missing values
X_baseline_2 = X[X_columns_2]   # Data Frame of features with <= 25% missing values threshold

In [30]:
X_baseline_1

,Systolic Blood Pressure,Diastolic Blood Pressure,"Hyperlipidemia (No=0, Yes=1)","Gender (Female=1, Male=2)","Diyabetes Mellitus (No=0, Yes=1)","Hypertension (No=0, Yes=1)",Body Mass Index,NFS,"Metabolic syndrome (No=0, Yes=1)",AST,ALT,Albumin,Total Cholesterol,FIB-4,Leukocyte,Hemoglobin,Trombosit,Age
0,135,77,1,1,0,1,35.56,-0.557850,0,27.0,49,5.00,170,1.066491,7300.0,12.7,217.0,60
1,120,72,0,1,0,0,30.78,-2.290680,0,17.0,18,4.40,170,0.801388,6000.0,12.9,275.0,55
2,120,80,1,2,1,1,34.95,-0.647403,1,51.0,74,5.10,190,1.532768,7700.0,13.5,205.0,53
3,124,83,1,2,1,0,31.02,-3.661870,1,31.0,72,4.90,199,0.352520,7300.0,15.1,342.0,33
4,124,79,0,2,0,0,25.91,-3.995284,0,32.0,51,5.30,211,0.490765,7000.0,17.4,210.0,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,140,100,1,1,0,1,37.78,-2.030823,1,138.0,189,4.20,218,1.890277,7400.0,13.6,308.0,58
601,120,80,1,2,0,1,28.61,-1.607660,1,36.0,55,4.90,283,1.136866,7890.0,14.7,269.0,63
602,150,80,1,1,1,0,40.30,-2.295062,1,30.0,39,4.33,183,0.541972,4600.0,9.1,390.0,44
603,150,80,1,1,1,0,40.35,-2.270562,1,30.0,39,4.30,183,0.541972,4600.0,9.1,390.0,44


In [31]:
X_baseline_2

,Total Protein,LDH,Hemoglobin - A1C,Waist Circumference,"Smoking Status (Never Smoked=1, Left Smoking=2, Smoking=3)",Ferritin,Mean Platelet Volume,Hip Circumference,Glucose,GGT,...,"Metabolic syndrome (No=0, Yes=1)",AST,ALT,Albumin,Total Cholesterol,FIB-4,Leukocyte,Hemoglobin,Trombosit,Age
0,7.70,281.0,NaN,119.0,2.0,524.0,9.20,104.0,119.0,19.0,...,0,27.0,49,5.00,170,1.066491,7300.0,12.7,217.0,60
1,7.20,219.0,NaN,101.0,1.0,41.3,8.10,98.0,NaN,20.0,...,0,17.0,18,4.40,170,0.801388,6000.0,12.9,275.0,55
2,8.40,263.0,4.70,116.0,2.0,294.0,9.30,112.0,93.0,53.0,...,1,51.0,74,5.10,190,1.532768,7700.0,13.5,205.0,53
3,7.60,166.0,4.70,114.0,2.0,46.0,7.10,114.0,90.0,49.0,...,1,31.0,72,4.90,199,0.352520,7300.0,15.1,342.0,33
4,7.90,376.0,4.82,101.0,2.0,158.0,7.40,97.0,89.0,34.0,...,0,32.0,51,5.30,211,0.490765,7000.0,17.4,210.0,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,7.90,264.0,5.50,110.0,1.0,24.1,8.70,108.0,91.0,46.0,...,1,138.0,189,4.20,218,1.890277,7400.0,13.6,308.0,58
601,NaN,195.0,5.80,108.0,2.0,56.0,8.75,117.0,115.0,65.0,...,1,36.0,55,4.90,283,1.136866,7890.0,14.7,269.0,63
602,7.59,227.0,6.60,127.0,1.0,44.0,7.90,108.0,133.0,29.0,...,1,30.0,39,4.33,183,0.541972,4600.0,9.1,390.0,44
603,7.60,227.0,6.60,127.0,1.0,44.0,7.90,108.0,133.0,29.0,...,1,30.0,39,4.30,183,0.541972,4600.0,9.1,390.0,44


In [ ]:
X_baseline_2 = fast_knn(X_baseline_2, k=30)
X_baseline_2.columns=X_columns_2